# Neural Network Model Testing

This notebook tests the neural network model for sentiment prediction on news headlines.

In [1]:
# Import necessary libraries
import sys
import os
import pandas as pd
from pathlib import Path

# Add project root to path for imports
sys.path.append(os.path.abspath('../..'))

# Import project modules
from src.models.predict_model import ModelPredictor
from src.models.train_model import ModelTrainer
from src.config import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
d:\Documents\CODE\HCMUT\Machine Learning Assignment\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Initialize the Model Predictor

We'll initialize the ModelPredictor class that will use our trained neural network model.

In [16]:
models = None
# model_dir = Path(MODEL_DIR)
# models = list(model_dir.glob('*rnn_*.pkl'))
# if not models:
#     model_dir = Path(EXPERIMENT_DIR)
#     models = list(model_dir.glob('*rnn_*.pkl'))
if not models:
    trainer = ModelTrainer()
    trainer.train_neural_network(batch_size=8)

# Initialize the predictor
predictor = ModelPredictor()

Starting RNN model training...
Class weights for balancing: {0: 1.0097222222222222, 1: 0.9807095642789694, 2: 1.0101431151868834}
Epoch 1/20
727/727 ━━━━━━━━━━━━━━━━━━━━ 27s 29ms/step - accuracy: 0.6178 - loss: 0.8478 - val_accuracy: 0.8693 - val_loss: 0.4358 - learning_rate: 0.0010
Epoch 2/20
727/727 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.8732 - loss: 0.3673 - val_accuracy: 0.8686 - val_loss: 0.3927 - learning_rate: 0.0010
Epoch 3/20
727/727 ━━━━━━━━━━━━━━━━━━━━ 22s 30ms/step - accuracy: 0.9223 - loss: 0.2605 - val_accuracy: 0.8652 - val_loss: 0.3726 - learning_rate: 0.0010
Epoch 4/20
727/727 ━━━━━━━━━━━━━━━━━━━━ 23s 31ms/step - accuracy: 0.9415 - loss: 0.2005 - val_accuracy: 0.8666 - val_loss: 0.4235 - learning_rate: 0.0010
Epoch 5/20
727/727 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.9579 - loss: 0.1514 - val_accuracy: 0.8604 - val_loss: 0.4659 - learning_rate: 0.0010
Epoch 6/20
727/727 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.9641 - loss: 0.1244 - val_accura

## 2. Test Single Headline Prediction

Let's test the model on a single headline first to check if everything is working.

In [29]:
# Test with a single positive headline
test_headline = "Company profits exceed expectations in Q1 2025"
result = predictor.predict_neural_network(test_headline)

# Display the result
if result:
    r = result[0]  # Get the first result
    print(f"\nHeadline: {r['headline']}")
    print(f"Predicted Sentiment: {r['sentiment']}")
    print(f"Confidence: {r['confidence']:.2f}")
    
    print("\nAll Probabilities:")
    for sentiment, prob in r['probabilities'].items():
        print(f"- {sentiment}: {prob:.2f}")
else:
    print("Prediction failed or no model found.")

Using most recent model: d:\Documents\CODE\HCMUT\Machine Learning Assignment\models\trained\rnn_076.pkl
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
Prediction class distribution:
negative (class 0): 0 predictions
neutral (class 1): 1 predictions
positive (class 2): 0 predictions

Headline: Company profits exceed expectations in Q1 2025
Predicted Sentiment: neutral
Confidence: 0.93

All Probabilities:
- negative: 0.02
- neutral: 0.93
- positive: 0.05


## 3. Test Multiple Headlines

Now let's test the model on multiple headlines with different expected sentiments.

In [30]:
# Test with multiple headlines
test_headlines = [
    "Stock market reaches all-time high as investor confidence grows",
    "Major company announces significant layoffs due to economic downturn",
    "Global trade continues at steady pace despite mild fluctuations",
    "Tech giant releases new product line with innovative features",
    "Retail sales decline for third consecutive quarter"
]

results = predictor.predict_neural_network(test_headlines)

# Display the results
if results:
    for r in results:
        print(f"Headline: {r['headline']}")
        print(f"Predicted Sentiment: {r['sentiment']} (confidence: {r['confidence']:.2f})")
        print()
else:
    print("Prediction failed or no model found.")

Using most recent model: d:\Documents\CODE\HCMUT\Machine Learning Assignment\models\trained\rnn_076.pkl


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step
Prediction class distribution:
negative (class 0): 2 predictions
neutral (class 1): 3 predictions
positive (class 2): 0 predictions
Headline: Stock market reaches all-time high as investor confidence grows
Predicted Sentiment: neutral (confidence: 0.60)

Headline: Major company announces significant layoffs due to economic downturn
Predicted Sentiment: negative (confidence: 0.99)

Headline: Global trade continues at steady pace despite mild fluctuations
Predicted Sentiment: neutral (confidence: 0.87)

Headline: Tech giant releases new product line with innovative features
Predicted Sentiment: neutral (confidence: 0.90)

Headline: Retail sales decline for third consecutive quarter
Predicted Sentiment: negative (confidence: 0.99)



## 4. Test on Real Dataset

Let's load a sample of the test dataset and predict sentiments.

In [31]:
# Load test dataset
test_data_path = os.path.join(EXTERNAL_DATA_PATH, "test_data.csv")
test_df = pd.read_csv(test_data_path)

print(f"Loaded test data with {len(test_df)} headlines")

# Show a few examples
test_df.head(100)


Loaded test data with 841 headlines


,Sentiment,News Headline
0,positive,"Nokia reports strong Q4 earnings, smartphone s..."
1,positive,Finnish telecom giant Elisa sees 12% revenue g...
2,positive,UPM-Kymmene's profit jumps 18% in latest quart...
3,negative,Metso announces closure of manufacturing plant...
4,positive,Kone wins major elevator contract for Dubai sk...
...,...,...
95,positive,Kemira chemicals reports stronger than expecte...
96,negative,Atria food company reports margin pressure due...
97,neutral,Huhtamaki packaging materials completes factor...
98,positive,UPM paper manufacturer wins sustainability awa...


In [32]:
# Make predictions
headlines = test_df['News Headline'].tolist()
results = predictor.predict_neural_network(headlines)

# Create a dataframe with predictions
if results:
    predicted_sentiments = [r['sentiment'] for r in results]
    confidence_scores = [round(r['confidence'], 2) for r in results]
    
    # Add predictions to the dataframe
    results_df = test_df.copy()
    results_df = results_df.rename(columns={'Sentiment': 'Actual Sentiment'})
    results_df['Predicted Sentiment'] = predicted_sentiments
    results_df['Confidence'] = confidence_scores
    
    print(f"Predictions completed")
else:
    print("Prediction failed or no model found.")

# Show some results
results_df.head(3)

Using most recent model: d:\Documents\CODE\HCMUT\Machine Learning Assignment\models\trained\rnn_076.pkl
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
Prediction class distribution:
negative (class 0): 262 predictions
neutral (class 1): 335 predictions
positive (class 2): 244 predictions
Predictions completed


,Actual Sentiment,News Headline,Predicted Sentiment,Confidence
0,positive,"Nokia reports strong Q4 earnings, smartphone s...",positive,0.93
1,positive,Finnish telecom giant Elisa sees 12% revenue g...,positive,0.99
2,positive,UPM-Kymmene's profit jumps 18% in latest quart...,positive,0.61


## 5. Model Evaluation

Let's evaluate the model's performance on the test dataset.

In [33]:
from sklearn.metrics import classification_report

# Calculate metrics
y_true = results_df['Actual Sentiment']
y_pred = results_df['Predicted Sentiment']

# Print classification report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

    negative       0.76      0.77      0.76       261
     neutral       0.66      0.84      0.74       263
    positive       0.90      0.69      0.78       317

    accuracy                           0.76       841
   macro avg       0.77      0.77      0.76       841
weighted avg       0.78      0.76      0.76       841



In [34]:
true = results_df['Predicted Sentiment'] == results_df['Actual Sentiment']
true.value_counts()

True     640
False    201
Name: count, dtype: int64